In [ ]:
#importing libraries
import pandas as pd
import tqdm as tqdm

In [ ]:
#reading in the data
stats = pd.read_csv("all_stats.csv")

In [79]:
#
stats

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,TOV%,USG%,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP
0,A.C. Green,SF,32,PHO,82,36,25.8,2.6,5.4,0.484,...,12.6,13.2,2.7,1.7,4.4,0.100,-0.4,-0.9,-1.3,0.4
1,A.C. Green,PF,37,MIA,82,1,17.2,1.8,4.0,0.444,...,10.8,14.4,1.1,2.1,3.2,0.110,-2.2,0.0,-2.2,-0.1
2,A.C. Green,PF,33,DAL,83,73,30.0,2.8,5.8,0.483,...,11.5,11.8,2.4,2.4,4.8,0.093,-1.0,-0.4,-1.4,0.4
3,A.C. Green,PF,34,DAL,82,68,32.3,3.0,6.5,0.453,...,10.1,11.6,2.8,2.4,5.2,0.095,-1.1,-0.3,-1.4,0.4
4,A.C. Green,PF,35,DAL,50,35,18.5,2.2,5.1,0.422,...,6.4,14.8,0.7,0.7,1.4,0.074,-1.1,-1.1,-2.2,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16028,Željko Rebrača,C,30,DET,30,12,16.3,2.7,4.8,0.552,...,14.9,19.2,0.7,0.7,1.3,0.133,-2.8,0.2,-2.6,-0.1
16029,Željko Rebrača,C,29,DET,74,4,15.9,2.6,5.1,0.505,...,15.7,21.4,1.4,1.9,3.3,0.134,-2.9,0.6,-2.3,-0.1
16030,Željko Rebrača,C,32,LAC,58,2,16.0,2.3,4.0,0.568,...,15.3,16.0,1.4,0.9,2.4,0.122,-1.7,0.4,-1.3,0.2
16031,Željko Rebrača,C,33,LAC,29,2,14.2,1.8,3.3,0.542,...,17.4,15.5,0.2,0.5,0.7,0.086,-3.7,0.9,-2.8,-0.1


In [81]:
#fill in missing values with 0
stats = stats.fillna(0)

In [82]:
stats.columns

Index(['Player', 'Pos', 'Age', 'Tm', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '3P',
       '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 'ORB',
       'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'Year',
       'Pts Won', 'Pts Max', 'Share', 'Team', 'W', 'L', 'W/L%', 'GB', 'PS/G',
       'PA/G', 'SRS', 'Rk', 'PER', 'TS%', '3PAr', 'FTr', 'ORB%', 'DRB%',
       'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%', 'OWS', 'DWS', 'WS',
       'WS/48', 'OBPM', 'DBPM', 'BPM', 'VORP'],
      dtype='object')

In [ ]:
#setting up the predictors
predictors = ['Age', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '3P',
       '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 'ORB',
       'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'Year',
       'W', 'L', 'W/L%', 'GB', 'PS/G',
       'PA/G', 'SRS', 'Rk', 'PER', 'TS%', '3PAr', 'FTr', 'ORB%', 'DRB%',
       'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%', 'OWS', 'DWS', 'WS',
       'WS/48', 'OBPM', 'DBPM', 'BPM', 'VORP']

In [99]:
#split data into training and testing sets
train = stats[stats["Year"] < 2025]
test = stats[stats["Year"] == 2025]

In [104]:
combination = pd.concat([test[["Player", "Share"]], predictions], axis=1)

In [105]:
combination.sort_values("Share", ascending=False).head(10)

,Player,Share,predictions
13676,Shai Gilgeous-Alexander,0.913,0.196172
11649,Nikola Jokić,0.787,0.234807
5675,Giannis Antetokounmpo,0.470,0.241553
7286,Jayson Tatum,0.311,0.116612
4478,Donovan Mitchell,0.074,0.052700
9670,LeBron James,0.016,0.109183
821,Anthony Edwards,0.012,0.085500
2051,Cade Cunningham,0.012,0.089100
14034,Stephen Curry,0.002,0.073662
6816,James Harden,0.001,0.073795


In [106]:
from sklearn.metrics import mean_squared_error

mean_squared_error(combination["Share"], combination["predictions"])

0.002399142289578565

In [107]:
#Sorting the combination dataframe by Share
combination = combination.sort_values("Share", ascending=False)
combination["Rk"] = list(range(1, combination.shape[0] + 1))


In [108]:
combination.head(10)

,Player,Share,predictions,Rk
13676,Shai Gilgeous-Alexander,0.913,0.196172,1
11649,Nikola Jokić,0.787,0.234807,2
5675,Giannis Antetokounmpo,0.470,0.241553,3
7286,Jayson Tatum,0.311,0.116612,4
4478,Donovan Mitchell,0.074,0.052700,5
9670,LeBron James,0.016,0.109183,6
821,Anthony Edwards,0.012,0.085500,7
2051,Cade Cunningham,0.012,0.089100,8
14034,Stephen Curry,0.002,0.073662,9
6816,James Harden,0.001,0.073795,10


In [109]:
#Sorting the combination dataframe by predictions
combination = combination.sort_values("predictions", ascending=False)
combination["Predicted_Rk"] = list(range(1, combination.shape[0] + 1))



In [110]:
combination.head(10)

,Player,Share,predictions,Rk,Predicted_Rk
5675,Giannis Antetokounmpo,0.470,0.241553,3,1
11649,Nikola Jokić,0.787,0.234807,2,2
13676,Shai Gilgeous-Alexander,0.913,0.196172,1,3
7286,Jayson Tatum,0.311,0.116612,4,4
9670,LeBron James,0.016,0.109183,6,5
4393,Domantas Sabonis,0.000,0.090086,130,6
2051,Cade Cunningham,0.012,0.089100,8,7
821,Anthony Edwards,0.012,0.085500,7,8
429,Alperen Şengün,0.000,0.082036,481,9
6816,James Harden,0.001,0.073795,10,10


In [111]:
combination.sort_values("Share", ascending=False).head(10)

,Player,Share,predictions,Rk,Predicted_Rk
13676,Shai Gilgeous-Alexander,0.913,0.196172,1,3
11649,Nikola Jokić,0.787,0.234807,2,2
5675,Giannis Antetokounmpo,0.470,0.241553,3,1
7286,Jayson Tatum,0.311,0.116612,4,4
4478,Donovan Mitchell,0.074,0.052700,5,23
9670,LeBron James,0.016,0.109183,6,5
821,Anthony Edwards,0.012,0.085500,7,8
2051,Cade Cunningham,0.012,0.089100,8,7
14034,Stephen Curry,0.002,0.073662,9,11
6816,James Harden,0.001,0.073795,10,10


In [112]:
def find_ap(combination):
    actual = combination.sort_values("Share", ascending=False).head(5)
    predicted = combination.sort_values("predictions", ascending=False)
    ps=[]
    found=0
    seen=1
    for index, row in predicted.iterrows():
        if row["Player"] in actual["Player"].values:
            found +=1
            ps.append(found/seen)
        seen +=1
    return sum(ps)/len(ps)



In [113]:
find_ap(combination)

0.8434782608695652

In [114]:
years = list(range(1991, 2025))

In [115]:
aps = []
all_predictions = []
for year in tqdm.tqdm(years[5:]):
    train = stats[stats["Year"] < year]
    test = stats[stats["Year"] == year]
    reg.fit(train[predictors], train["Share"])
    predictions = reg.predict((test[predictors]))
    predictions = pd.DataFrame(predictions, columns=["predictions"], index=test.index)
    combination = pd.concat([test[["Player", "Share"]], predictions], axis=1)
    all_predictions.append(combination)
    aps.append(find_ap(combination))

100%|██████████| 29/29 [00:04<00:00,  6.90it/s]


In [116]:
sum(aps)/len(aps)

0.7831841474512833

In [117]:
def add_ranks(combination):
    # Add actual ranks
    combination = combination.sort_values("Share", ascending=False)
    combination["Rk"] = list(range(1, combination.shape[0] + 1))
    # Add predicted ranks
    combination = combination.sort_values("predictions", ascending=False)
    combination["Predicted_Rk"] = list(range(1, combination.shape[0] + 1))
    combination["Diff"] = combination["Rk"] - combination["Predicted_Rk"]
    return combination


In [118]:
ranking = add_ranks(all_predictions[1])
ranking[ranking["Rk"] < 6].sort_values("Diff", ascending=False)

,Player,Share,predictions,Rk,Predicted_Rk,Diff
10939,Michael Jordan,0.832,0.267267,2,1,1
5781,Grant Hill,0.327,0.220931,3,3,0
8588,Karl Malone,0.857,0.252652,1,2,-1
14610,Tim Hardaway,0.207,0.132227,4,8,-4
5700,Glen Rice,0.117,0.023543,5,82,-77


In [138]:
#Backtesting the model
def backtest(stats, model, year, predictors):
    aps = []
    all_predictions = []
    for year in tqdm.tqdm(years[5:]):
        train = stats[stats["Year"] < year]
        test = stats[stats["Year"] == year]

        # Convert to numpy arrays
        X_train = train[predictors].values
        y_train = train["Share"].values
        X_test = test[predictors].values

        model.fit(train[predictors], train["Share"])
        predictions = model.predict((test[predictors]))
        predictions = pd.DataFrame(predictions, columns=["predictions"], index=test.index)
        combination = pd.concat([test[["Player", "Share"]], predictions], axis=1)
        combination = add_ranks(combination)
        all_predictions.append(combination)
        aps.append(find_ap(combination))
    return sum(aps)/len(aps), aps, pd.concat(all_predictions)
    

In [120]:
mean_ap, aps, all_predictions = backtest(stats, reg, years[5:], predictors)

100%|██████████| 29/29 [00:04<00:00,  6.69it/s]


In [121]:
mean_ap

0.7831841474512833

In [123]:
#sorting predictions by difference between predicted and actual rank
all_predictions[all_predictions["Rk"] <= 5].sort_values("Diff").head(10)

,Player,Share,predictions,Rk,Predicted_Rk,Diff
7130,Jason Kidd,0.135,-0.015134,5,203,-198
5700,Glen Rice,0.117,0.023543,5,82,-77
14156,Steve Nash,0.839,0.033841,1,48,-47
12132,Peja Stojaković,0.228,0.032993,4,49,-45
14151,Steve Nash,0.739,0.054801,1,34,-33
7123,Jason Kidd,0.712,0.047301,2,33,-31
14152,Steve Nash,0.785,0.073830,2,21,-19
2658,Chris Paul,0.138,0.050902,5,23,-18
2420,Chauncey Billups,0.344,0.081760,5,22,-17
14039,Stephen Curry,0.173,0.073278,5,21,-16


In [125]:
#sorting the coefficients by their magnitude
pd.concat([pd.Series(reg.coef_), pd.Series(predictors)], axis=1).sort_values(0, ascending=False)

,0,1
13,0.166277,eFG%
29,0.077878,W/L%
18,0.030230,DRB
54,0.024552,VORP
15,0.023446,FTA
16,0.020774,FT%
5,0.014550,FGA
23,0.013164,TOV
50,0.012217,WS/48
17,0.009961,ORB


In [ ]:
#adding ratios to the stats dataframe
stat_ratios = stats[["PTS", "AST", "STL", "BLK", "3P", "Year"]].groupby("Year").apply(lambda x: x/x.mean())
stat_ratios = stat_ratios.reset_index(level=0, drop=True)
stats[["PTS_T", "AST_R", "STL_R", "BLK_R", "3P_R"]] = stat_ratios[["PTS", "AST", "STL", "BLK", "3P"]]

C:\Users\Daniel\AppData\Local\Temp\ipykernel_22648\2022889540.py:1: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  stat_ratios = stats[["PTS", "AST", "STL", "BLK", "3P", "Year"]].groupby("Year").apply(lambda x: x/x.mean())


In [132]:
stats.to_csv("all_stats.csv", index=False)

In [133]:
predictors += ["PTS_T", "AST_R", "STL_R", "BLK_R", "3P_R"]

In [48]:
from sklearn.ensemble import RandomForestRegressor
#Random Forest Regressor
rf = RandomForestRegressor(n_estimators= 50, random_state=1, min_samples_split=5)



In [49]:
#mean_ap, aps, all_predictions = backtest(stats, rf, years[28:], predictors)

In [159]:
from sklearn.ensemble import HistGradientBoostingRegressor

hgb = HistGradientBoostingRegressor(random_state=1, max_iter=100)

mean_ap, aps, all_predictions = backtest(stats, hgb, years[5:], predictors)


100%|██████████| 29/29 [00:48<00:00,  1.66s/it]


In [161]:
mean_ap

0.7914464689651651

In [163]:
# See if you're getting better or worse over time
yearly_performance = pd.DataFrame({'Year': years[5:], 'AP': aps})
print(yearly_performance.sort_values('AP'))

    Year        AP
3   1999  0.156683
9   2005  0.423452
6   2002  0.555917
5   2001  0.636667
16  2012  0.641039
8   2004  0.673372
21  2017  0.686190
0   1996  0.697778
15  2011  0.741667
1   1997  0.762273
4   2000  0.783333
12  2008  0.786923
10  2006  0.788889
25  2021  0.843478
23  2019  0.844444
11  2007  0.852857
24  2020  0.852857
18  2014  0.855556
19  2015  0.876667
7   2003  0.883333
2   1998  0.902857
17  2013  0.902857
27  2023  0.926667
28  2024  0.942857
26  2022  0.966667
22  2018  0.966667
14  2010  1.000000
13  2009  1.000000
20  2016  1.000000


In [170]:
# Assuming you have existing results
yearly_performance = pd.DataFrame({'Year': years[5:], 'AP': aps})

# Filter to recent years
recent_performance = yearly_performance[yearly_performance['Year'] >= 2019]
recent_ap = recent_performance['AP'].mean()

print(f"AP for years >= 2010: {recent_ap:.4f}")
print(f"Number of years: {len(recent_performance)}")

AP for years >= 2010: 0.9313
Number of years: 6


In [164]:
from sklearn.ensemble import HistGradientBoostingRegressor, RandomForestRegressor, GradientBoostingRegressor, VotingRegressor
rf_model = RandomForestRegressor(n_estimators=100, random_state=1)
gb_model = GradientBoostingRegressor(n_estimators=100, random_state=1)
hgb_model = HistGradientBoostingRegressor(random_state=1, max_iter=100)

voting_model = VotingRegressor(estimators=[('rf', rf_model), ('gb', gb_model), ('hgb', hgb_model)])

mean_ap, aps, all_predictions = backtest(stats, voting_model, years[5:], predictors)

100%|██████████| 29/29 [40:33<00:00, 83.93s/it] 


In [165]:
mean_ap

0.8096465876412737

In [166]:
combination.sort_values("Share", ascending=False).head(10)

,Player,Share,predictions
11645,Nikola Jokić,0.935,0.255345
13672,Shai Gilgeous-Alexander,0.646,0.154796
9945,Luka Dončić,0.572,0.209183
5666,Giannis Antetokounmpo,0.194,0.230050
6626,Jalen Brunson,0.143,0.117840
7285,Jayson Tatum,0.087,0.103313
823,Anthony Edwards,0.018,0.072423
4394,Domantas Sabonis,0.003,0.142944
8939,Kevin Durant,0.001,0.090558
144,Admiral Schofield,0.000,0.004341


In [168]:
combination = combination.sort_values("Share", ascending=False)
combination["Rk"] = list(range(1, combination.shape[0] + 1))
combination = combination.sort_values("predictions", ascending=False)
combination["Predicted_Rk"] = list(range(1, combination.shape[0] + 1))

In [169]:
combination

,Player,Share,predictions,Rk,Predicted_Rk
11645,Nikola Jokić,0.935,0.255345,1,1
5666,Giannis Antetokounmpo,0.194,0.230050,4,2
9945,Luka Dončić,0.572,0.209183,3,3
13672,Shai Gilgeous-Alexander,0.646,0.154796,2,4
4394,Domantas Sabonis,0.003,0.142944,8,5
...,...,...,...,...,...
13308,Saddiq Bey,0.000,-0.051221,428,490
7479,Jeremy Sochan,0.000,-0.053867,244,491
11251,Mitchell Robinson,0.000,-0.054046,367,492
8219,Jordan Poole,0.000,-0.057382,268,493
